In [230]:
import requests
import pandas as pd
from datetime import datetime
import plotly.express as px
import numpy as np

# UNI

Посмотрим изменение цены и объемов UNI по отношению к USD за максимальный отрезок времени. 

In [313]:
uni_market_chart = requests.get(
    "https://api.coingecko.com/api/v3/coins/uniswap/market_chart?vs_currency=usd&days=max").json()

In [233]:
uni_market_chart.keys()

dict_keys(['prices', 'market_caps', 'total_volumes'])

In [234]:
uni_prices_df = pd.DataFrame({
    'date': [datetime.utcfromtimestamp(value[0] / 1000) for value in uni_market_chart['prices']], 
    'prices': [value[1] for value in uni_market_chart['prices']]
})

In [239]:
uni_total_supply_df = pd.DataFrame({
    'date': [datetime.utcfromtimestamp(value[0] / 1000) for value in uni_market_chart['total_volumes']], 
    'total_supply': [market_cap[1] / price[1] for price, market_cap in zip(uni_market_chart['prices'], 
                                                                    uni_market_chart['total_volumes'])]
})

Сравним с биткоином как с некоторым бенчамрком рынка.

In [338]:
btc_market_chart = requests.get(
    "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=max").json()
btc_market_df = pd.DataFrame({
    'date': [datetime.utcfromtimestamp(value[0] / 1000) for value in btc_market_chart['prices']], 
    'prices': [value[1] for value in btc_market_chart['prices']]
})

In [240]:
px.line(uni_prices_df, x='date', y='prices', title='UNI prices')

In [352]:
px.line(btc_market_df[btc_market_df.date > min(uni_prices_df['date'])], x='date', y='prices', title='BTC prices')

На глаз, ЮНИ довольно скореллирован с битокоином и рынком криптовалют в целом. У него какая-нибудь положительная бета к рынку с модулем в районе 0.75 (на глазок)

In [241]:
px.line(uni_total_supply_df, x='date', y='total_supply', title='UNI total supply')

Ну тут немножко не бьется с https://uniswap.org/images/posts/uni/Release_Schedule.png Но в целом понятно. Сначала созадли под миллиард юни токенов, а потом часть из них стала недоступна и будет становится доступной со временем (хотя роста их количества не видно)  В общем, тут есть вопросы. 

# Uniswap TVL

Рассмотрим изменение TVL на разных блокчейнах:

In [242]:
uniswap_json = requests.get('https://api.llama.fi/protocol/uniswap').json()

In [243]:
uniswap_json.keys()

dict_keys(['id', 'name', 'url', 'description', 'logo', 'gecko_id', 'cmcId', 'chains', 'twitter', 'currentChainTvls', 'chainTvls', 'tokens', 'tokensInUsd', 'tvl', 'isParentProtocol', 'otherProtocols', 'hallmarks'])

In [244]:
uniswap_json['chainTvls'].keys()

dict_keys(['Ethereum', 'Optimism', 'Polygon', 'Celo', 'Arbitrum'])

In [245]:
tvls = uniswap_json['chainTvls']

In [246]:
tvls_df = None
for i, key in enumerate(tvls.keys()):
    tmp = pd.DataFrame(tvls[key]['tvl']).rename(columns={'totalLiquidityUSD': key})
    if i == 0:
        tvls_df = tmp
    else: 
        tvls_df = tvls_df.merge(tmp, on='date', suffixes=('', key), how='outer')
    
tvls_df['date'] = tvls_df.date.apply(lambda x: datetime.utcfromtimestamp(x))

In [247]:
tvls_df

,date,Ethereum,Optimism,Polygon,Celo,Arbitrum
0,2018-11-03,3.464389e+04,NaN,NaN,NaN,NaN
1,2018-11-04,4.126818e+04,NaN,NaN,NaN,NaN
2,2018-11-05,4.055178e+04,NaN,NaN,NaN,NaN
3,2018-11-06,4.309134e+04,NaN,NaN,NaN,NaN
4,2018-11-07,1.539794e+05,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1457,2022-10-30,3.994893e+09,5.039911e+07,8.913595e+07,9.288965e+05,7.081866e+07
1458,2022-10-31,4.035889e+09,5.028660e+07,8.859798e+07,9.231935e+05,7.014237e+07
1459,2022-11-01,4.006696e+09,4.964292e+07,8.898412e+07,1.064409e+06,7.107238e+07
1460,2022-11-02,4.034659e+09,4.914678e+07,9.036235e+07,9.278947e+05,7.164081e+07


In [248]:
tvls_df.fillna(0)

,date,Ethereum,Optimism,Polygon,Celo,Arbitrum
0,2018-11-03,3.464389e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,2018-11-04,4.126818e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
2,2018-11-05,4.055178e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,2018-11-06,4.309134e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
4,2018-11-07,1.539794e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...
1457,2022-10-30,3.994893e+09,5.039911e+07,8.913595e+07,9.288965e+05,7.081866e+07
1458,2022-10-31,4.035889e+09,5.028660e+07,8.859798e+07,9.231935e+05,7.014237e+07
1459,2022-11-01,4.006696e+09,4.964292e+07,8.898412e+07,1.064409e+06,7.107238e+07
1460,2022-11-02,4.034659e+09,4.914678e+07,9.036235e+07,9.278947e+05,7.164081e+07


In [288]:
px.line(tvls_df[tvls_df.date > '2022-01-01'], x='date', y=[x for x in tvls_df.columns if x != 'date'], 
      title='Uniswap TVL')

Основная сеть в Юнисвопе - эфириум, она гораздо больше по TVL. Ну и снижение объемов есть по мере падения рынка и общего объема торгов. 

Посмотрим на TVL без Ethereum поближе.

In [308]:
px.line(tvls_df.drop('Ethereum', axis=1)[tvls_df.date > '2022-01-01'], x='date', 
        y=[x for x in tvls_df.drop('Ethereum', axis=1).columns if x != 'date'], title='Uniswap TVL')

## Uniswap V2 & Uniswap V3



Соберем все в одну функцию и посмотрим изменение TVL отдельно для Uniswap V2 и Uniswap V3. 

In [285]:
def get_TVL_plot(url, title, start_date):
    uniswap_json = requests.get(url).json()

    tvls = uniswap_json['chainTvls']

    tvls_df = None
    for i, key in enumerate(tvls.keys()):
        tmp = pd.DataFrame(tvls[key]['tvl']).rename(columns={'totalLiquidityUSD': key})
        if i == 0:
            tvls_df = tmp
        else: 
            tvls_df = tvls_df.merge(tmp, on='date', suffixes=('', key), how='outer')

    tvls_df['date'] = tvls_df.date.apply(lambda x: datetime.utcfromtimestamp(x))

    tvls_df.fillna(0)

    return px.line(tvls_df[tvls_df.date > start_date], x='date', y=[x for x in tvls_df.columns if x != 'date'], 
          title=title)

In [286]:
get_TVL_plot('https://api.llama.fi/protocol/uniswap-v2', 'Uniswap V2 TVL', '2021-01-01')

In [287]:
get_TVL_plot('https://api.llama.fi/protocol/uniswap-v3', 'Uniswap V3 TVL', '2022-01-01')

Есть ощущение, что часть торгующих на юнисвоп2 перешли на юнисвоп3, есть ощущение перетока. 

# Uniswap Trading Volumes

In [282]:
def unisawapTradingVolumes(url, name):
    uniswap_volume_json = requests.get(url).json()
    uniswap_volume_df = pd.DataFrame({
        'date': [datetime.utcfromtimestamp(value[0] / 1000) for value in uniswap_volume_json], 
        'value': [float(value[1]) for value in uniswap_volume_json],
    })
    return px.line(uniswap_volume_df, x='date', y='value', title=name + ' Trading Volumes')

## Uniswap V2

In [283]:
unisawapTradingVolumes('https://api.coingecko.com/api/v3/exchanges/uniswap_v2/volume_chart?days=500', 'Uniswap V2')

## Uniswap V3

In [284]:
unisawapTradingVolumes('https://api.coingecko.com/api/v3/exchanges/uniswap_v3/volume_chart?days=500', 'Uniswap V3')

# Uniswap V3 Connector

In [310]:
import time

from abc import ABC, abstractmethod
from dataclasses import dataclass


@dataclass
class Token:
    """
    Token data structure
    """
    address: str
    decimals: str
    symbol: str
    chain: str
        

def symbol_to_token(symbol: str) -> Token:
    tokens = {
                'BUSD': Token('0xe9e7CEA3DedcA5984780Bafc599bD69ADd087D56', 18, 'BUSD', 'bsc'),
                'USDT': Token('0x55d398326f99059fF775485246999027B3197955', 18, 'BUSD', 'bsc'),
             }
    return tokens[symbol]

In [311]:
class DexConnector(ABC):
    """
    Abstact Dex Connector
    """

    def __init__(self, wallet_address: str, private_key: str, node: str, **kwargs):
        self.private_key = private_key
        self.wallet_address = wallet_address

    def _make_transaction(self, transaction, **kwargs):
        tx = transaction.buildTransaction({
                                            'nonce': self.w3.eth.getTransactionCount(self.wallet_address),
                                            'gas': kwargs.get('gas', 2000000),
                                            'gasPrice': int(5*kwargs.get('gasPrice', self.w3.eth.gasPrice + 1000000))
                                         })

        signed_tx = self.w3.eth.account.signTransaction(tx, private_key=self.private_key)
        return self.w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        
    @abstractmethod
    def make_market_order(self, ticker: str, side: str, quantity: float, **kwargs) -> dict:
        """
        make_market_order: make market order
        ticker: str Token0/Token1
        side: str BUY | SELL
        quantity: float size of Token0 to make side market order
        """
        raise NotImplementedError()
        
    @abstractmethod
    def get_price(self, ticker: str, **kwargs) -> float:
        """
        get_price: get market price using AMM
        ticker: str Token0/Token1
        """
        raise NotImplementedError()

In [366]:
class UniswapV3Connector(DexConnector):
    """
    UniswapV3-like connector interface
    """
    ABI = '[{"inputs":[{"internalType":"address","name":"_factoryV2","type":"address"},{"internalType":"address","name":"factoryV3","type":"address"},{"internalType":"address","name":"_positionManager","type":"address"},{"internalType":"address","name":"_WETH9","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"WETH9","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"}],"name":"approveMax","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"}],"name":"approveMaxMinusOne","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"}],"name":"approveZeroThenMax","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"}],"name":"approveZeroThenMaxMinusOne","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"bytes","name":"data","type":"bytes"}],"name":"callPositionManager","outputs":[{"internalType":"bytes","name":"result","type":"bytes"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"bytes[]","name":"paths","type":"bytes[]"},{"internalType":"uint128[]","name":"amounts","type":"uint128[]"},{"internalType":"uint24","name":"maximumTickDivergence","type":"uint24"},{"internalType":"uint32","name":"secondsAgo","type":"uint32"}],"name":"checkOracleSlippage","outputs":[],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"bytes","name":"path","type":"bytes"},{"internalType":"uint24","name":"maximumTickDivergence","type":"uint24"},{"internalType":"uint32","name":"secondsAgo","type":"uint32"}],"name":"checkOracleSlippage","outputs":[],"stateMutability":"view","type":"function"},{"inputs":[{"components":[{"internalType":"bytes","name":"path","type":"bytes"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMinimum","type":"uint256"}],"internalType":"struct IV3SwapRouter.ExactInputParams","name":"params","type":"tuple"}],"name":"exactInput","outputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"components":[{"internalType":"address","name":"tokenIn","type":"address"},{"internalType":"address","name":"tokenOut","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMinimum","type":"uint256"},{"internalType":"uint160","name":"sqrtPriceLimitX96","type":"uint160"}],"internalType":"struct IV3SwapRouter.ExactInputSingleParams","name":"params","type":"tuple"}],"name":"exactInputSingle","outputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"components":[{"internalType":"bytes","name":"path","type":"bytes"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"uint256","name":"amountInMaximum","type":"uint256"}],"internalType":"struct IV3SwapRouter.ExactOutputParams","name":"params","type":"tuple"}],"name":"exactOutput","outputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"components":[{"internalType":"address","name":"tokenIn","type":"address"},{"internalType":"address","name":"tokenOut","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"uint256","name":"amountInMaximum","type":"uint256"},{"internalType":"uint160","name":"sqrtPriceLimitX96","type":"uint160"}],"internalType":"struct IV3SwapRouter.ExactOutputSingleParams","name":"params","type":"tuple"}],"name":"exactOutputSingle","outputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"factory","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"factoryV2","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"getApprovalType","outputs":[{"internalType":"enum IApproveAndCall.ApprovalType","name":"","type":"uint8"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"components":[{"internalType":"address","name":"token0","type":"address"},{"internalType":"address","name":"token1","type":"address"},{"internalType":"uint256","name":"tokenId","type":"uint256"},{"internalType":"uint256","name":"amount0Min","type":"uint256"},{"internalType":"uint256","name":"amount1Min","type":"uint256"}],"internalType":"struct IApproveAndCall.IncreaseLiquidityParams","name":"params","type":"tuple"}],"name":"increaseLiquidity","outputs":[{"internalType":"bytes","name":"result","type":"bytes"}],"stateMutability":"payable","type":"function"},{"inputs":[{"components":[{"internalType":"address","name":"token0","type":"address"},{"internalType":"address","name":"token1","type":"address"},{"internalType":"uint24","name":"fee","type":"uint24"},{"internalType":"int24","name":"tickLower","type":"int24"},{"internalType":"int24","name":"tickUpper","type":"int24"},{"internalType":"uint256","name":"amount0Min","type":"uint256"},{"internalType":"uint256","name":"amount1Min","type":"uint256"},{"internalType":"address","name":"recipient","type":"address"}],"internalType":"struct IApproveAndCall.MintParams","name":"params","type":"tuple"}],"name":"mint","outputs":[{"internalType":"bytes","name":"result","type":"bytes"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"previousBlockhash","type":"bytes32"},{"internalType":"bytes[]","name":"data","type":"bytes[]"}],"name":"multicall","outputs":[{"internalType":"bytes[]","name":"","type":"bytes[]"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"bytes[]","name":"data","type":"bytes[]"}],"name":"multicall","outputs":[{"internalType":"bytes[]","name":"","type":"bytes[]"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"bytes[]","name":"data","type":"bytes[]"}],"name":"multicall","outputs":[{"internalType":"bytes[]","name":"results","type":"bytes[]"}],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"positionManager","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"pull","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"refundETH","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"selfPermit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"nonce","type":"uint256"},{"internalType":"uint256","name":"expiry","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"selfPermitAllowed","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"nonce","type":"uint256"},{"internalType":"uint256","name":"expiry","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"selfPermitAllowedIfNecessary","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"selfPermitIfNecessary","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"}],"name":"swapExactTokensForTokens","outputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"uint256","name":"amountInMax","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"}],"name":"swapTokensForExactTokens","outputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amountMinimum","type":"uint256"},{"internalType":"address","name":"recipient","type":"address"}],"name":"sweepToken","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amountMinimum","type":"uint256"}],"name":"sweepToken","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amountMinimum","type":"uint256"},{"internalType":"uint256","name":"feeBips","type":"uint256"},{"internalType":"address","name":"feeRecipient","type":"address"}],"name":"sweepTokenWithFee","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amountMinimum","type":"uint256"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"feeBips","type":"uint256"},{"internalType":"address","name":"feeRecipient","type":"address"}],"name":"sweepTokenWithFee","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"int256","name":"amount0Delta","type":"int256"},{"internalType":"int256","name":"amount1Delta","type":"int256"},{"internalType":"bytes","name":"_data","type":"bytes"}],"name":"uniswapV3SwapCallback","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountMinimum","type":"uint256"},{"internalType":"address","name":"recipient","type":"address"}],"name":"unwrapWETH9","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountMinimum","type":"uint256"}],"name":"unwrapWETH9","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountMinimum","type":"uint256"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"feeBips","type":"uint256"},{"internalType":"address","name":"feeRecipient","type":"address"}],"name":"unwrapWETH9WithFee","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountMinimum","type":"uint256"},{"internalType":"uint256","name":"feeBips","type":"uint256"},{"internalType":"address","name":"feeRecipient","type":"address"}],"name":"unwrapWETH9WithFee","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"value","type":"uint256"}],"name":"wrapETH","outputs":[],"stateMutability":"payable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
    nonFungiblePositionManagerABI = '[{"inputs": [{"internalType": "address","name": "_factory","type": "address"},{"internalType": "address","name": "_WETH9","type": "address"},{"internalType": "address","name": "_tokenDescriptor_","type": "address"}],"stateMutability": "nonpayable","type": "constructor"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "address","name": "owner","type": "address"},{"indexed": true,"internalType": "address","name": "approved","type": "address"},{"indexed": true,"internalType": "uint256","name": "tokenId","type": "uint256"}],"name": "Approval","type": "event"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "address","name": "owner","type": "address"},{"indexed": true,"internalType": "address","name": "operator","type": "address"},{"indexed": false,"internalType": "bool","name": "approved","type": "bool"}],"name": "ApprovalForAll","type": "event"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "uint256","name": "tokenId","type": "uint256"},{"indexed": false,"internalType": "address","name": "recipient","type": "address"},{"indexed": false,"internalType": "uint256","name": "amount0","type": "uint256"},{"indexed": false,"internalType": "uint256","name": "amount1","type": "uint256"}],"name": "Collect","type": "event"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "uint256","name": "tokenId","type": "uint256"},{"indexed": false,"internalType": "uint128","name": "liquidity","type": "uint128"},{"indexed": false,"internalType": "uint256","name": "amount0","type": "uint256"},{"indexed": false,"internalType": "uint256","name": "amount1","type": "uint256"}],"name": "DecreaseLiquidity","type": "event"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "uint256","name": "tokenId","type": "uint256"},{"indexed": false,"internalType": "uint128","name": "liquidity","type": "uint128"},{"indexed": false,"internalType": "uint256","name": "amount0","type": "uint256"},{"indexed": false,"internalType": "uint256","name": "amount1","type": "uint256"}],"name": "IncreaseLiquidity","type": "event"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "address","name": "from","type": "address"},{"indexed": true,"internalType": "address","name": "to","type": "address"},{"indexed": true,"internalType": "uint256","name": "tokenId","type": "uint256"}],"name": "Transfer","type": "event"},{"inputs": [],"name": "DOMAIN_SEPARATOR","outputs": [{"internalType": "bytes32","name": "","type": "bytes32"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "PERMIT_TYPEHASH","outputs": [{"internalType": "bytes32","name": "","type": "bytes32"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "WETH9","outputs": [{"internalType": "address","name": "","type": "address"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "address","name": "to","type": "address"},{"internalType": "uint256","name": "tokenId","type": "uint256"}],"name": "approve","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "address","name": "owner","type": "address"}],"name": "balanceOf","outputs": [{"internalType": "uint256","name": "","type": "uint256"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "baseURI","outputs": [{"internalType": "string","name": "","type": "string"}],"stateMutability": "pure","type": "function"},{"inputs": [{"internalType": "uint256","name": "tokenId","type": "uint256"}],"name": "burn","outputs": [],"stateMutability": "payable","type": "function"},{"inputs": [{"components": [{"internalType": "uint256","name": "tokenId","type": "uint256"},{"internalType": "address","name": "recipient","type": "address"},{"internalType": "uint128","name": "amount0Max","type": "uint128"},{"internalType": "uint128","name": "amount1Max","type": "uint128"}],"internalType": "struct INonfungiblePositionManager.CollectParams","name": "params","type": "tuple"}],"name": "collect","outputs": [{"internalType": "uint256","name": "amount0","type": "uint256"},{"internalType": "uint256","name": "amount1","type": "uint256"}],"stateMutability": "payable","type": "function"},{"inputs": [{"internalType": "address","name": "token0","type": "address"},{"internalType": "address","name": "token1","type": "address"},{"internalType": "uint24","name": "fee","type": "uint24"},{"internalType": "uint160","name": "sqrtPriceX96","type": "uint160"}],"name": "createAndInitializePoolIfNecessary","outputs": [{"internalType": "address","name": "pool","type": "address"}],"stateMutability": "payable","type": "function"},{"inputs": [{"components": [{"internalType": "uint256","name": "tokenId","type": "uint256"},{"internalType": "uint128","name": "liquidity","type": "uint128"},{"internalType": "uint256","name": "amount0Min","type": "uint256"},{"internalType": "uint256","name": "amount1Min","type": "uint256"},{"internalType": "uint256","name": "deadline","type": "uint256"}],"internalType": "struct INonfungiblePositionManager.DecreaseLiquidityParams","name": "params","type": "tuple"}],"name": "decreaseLiquidity","outputs": [{"internalType": "uint256","name": "amount0","type": "uint256"},{"internalType": "uint256","name": "amount1","type": "uint256"}],"stateMutability": "payable","type": "function"},{"inputs": [],"name": "factory","outputs": [{"internalType": "address","name": "","type": "address"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "uint256","name": "tokenId","type": "uint256"}],"name": "getApproved","outputs": [{"internalType": "address","name": "","type": "address"}],"stateMutability": "view","type": "function"},{"inputs": [{"components": [{"internalType": "uint256","name": "tokenId","type": "uint256"},{"internalType": "uint256","name": "amount0Desired","type": "uint256"},{"internalType": "uint256","name": "amount1Desired","type": "uint256"},{"internalType": "uint256","name": "amount0Min","type": "uint256"},{"internalType": "uint256","name": "amount1Min","type": "uint256"},{"internalType": "uint256","name": "deadline","type": "uint256"}],"internalType": "struct INonfungiblePositionManager.IncreaseLiquidityParams","name": "params","type": "tuple"}],"name": "increaseLiquidity","outputs": [{"internalType": "uint128","name": "liquidity","type": "uint128"},{"internalType": "uint256","name": "amount0","type": "uint256"},{"internalType": "uint256","name": "amount1","type": "uint256"}],"stateMutability": "payable","type": "function"},{"inputs": [{"internalType": "address","name": "owner","type": "address"},{"internalType": "address","name": "operator","type": "address"}],"name": "isApprovedForAll","outputs": [{"internalType": "bool","name": "","type": "bool"}],"stateMutability": "view","type": "function"},{"inputs": [{"components": [{"internalType": "address","name": "token0","type": "address"},{"internalType": "address","name": "token1","type": "address"},{"internalType": "uint24","name": "fee","type": "uint24"},{"internalType": "int24","name": "tickLower","type": "int24"},{"internalType": "int24","name": "tickUpper","type": "int24"},{"internalType": "uint256","name": "amount0Desired","type": "uint256"},{"internalType": "uint256","name": "amount1Desired","type": "uint256"},{"internalType": "uint256","name": "amount0Min","type": "uint256"},{"internalType": "uint256","name": "amount1Min","type": "uint256"},{"internalType": "address","name": "recipient","type": "address"},{"internalType": "uint256","name": "deadline","type": "uint256"}],"internalType": "struct INonfungiblePositionManager.MintParams","name": "params","type": "tuple"}],"name": "mint","outputs": [{"internalType": "uint256","name": "tokenId","type": "uint256"},{"internalType": "uint128","name": "liquidity","type": "uint128"},{"internalType": "uint256","name": "amount0","type": "uint256"},{"internalType": "uint256","name": "amount1","type": "uint256"}],"stateMutability": "payable","type": "function"},{"inputs": [{"internalType": "bytes[]","name": "data","type": "bytes[]"}],"name": "multicall","outputs": [{"internalType": "bytes[]","name": "results","type": "bytes[]"}],"stateMutability": "payable","type": "function"},{"inputs": [],"name": "name","outputs": [{"internalType": "string","name": "","type": "string"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "uint256","name": "tokenId","type": "uint256"}],"name": "ownerOf","outputs": [{"internalType": "address","name": "","type": "address"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "address","name": "spender","type": "address"},{"internalType": "uint256","name": "tokenId","type": "uint256"},{"internalType": "uint256","name": "deadline","type": "uint256"},{"internalType": "uint8","name": "v","type": "uint8"},{"internalType": "bytes32","name": "r","type": "bytes32"},{"internalType": "bytes32","name": "s","type": "bytes32"}],"name": "permit","outputs": [],"stateMutability": "payable","type": "function"},{"inputs": [{"internalType": "uint256","name": "tokenId","type": "uint256"}],"name": "positions","outputs": [{"internalType": "uint96","name": "nonce","type": "uint96"},{"internalType": "address","name": "operator","type": "address"},{"internalType": "address","name": "token0","type": "address"},{"internalType": "address","name": "token1","type": "address"},{"internalType": "uint24","name": "fee","type": "uint24"},{"internalType": "int24","name": "tickLower","type": "int24"},{"internalType": "int24","name": "tickUpper","type": "int24"},{"internalType": "uint128","name": "liquidity","type": "uint128"},{"internalType": "uint256","name": "feeGrowthInside0LastX128","type": "uint256"},{"internalType": "uint256","name": "feeGrowthInside1LastX128","type": "uint256"},{"internalType": "uint128","name": "tokensOwed0","type": "uint128"},{"internalType": "uint128","name": "tokensOwed1","type": "uint128"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "refundETH","outputs": [],"stateMutability": "payable","type": "function"},{"inputs": [{"internalType": "address","name": "from","type": "address"},{"internalType": "address","name": "to","type": "address"},{"internalType": "uint256","name": "tokenId","type": "uint256"}],"name": "safeTransferFrom","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "address","name": "from","type": "address"},{"internalType": "address","name": "to","type": "address"},{"internalType": "uint256","name": "tokenId","type": "uint256"},{"internalType": "bytes","name": "_data","type": "bytes"}],"name": "safeTransferFrom","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "address","name": "token","type": "address"},{"internalType": "uint256","name": "value","type": "uint256"},{"internalType": "uint256","name": "deadline","type": "uint256"},{"internalType": "uint8","name": "v","type": "uint8"},{"internalType": "bytes32","name": "r","type": "bytes32"},{"internalType": "bytes32","name": "s","type": "bytes32"}],"name": "selfPermit","outputs": [],"stateMutability": "payable","type": "function"},{"inputs": [{"internalType": "address","name": "token","type": "address"},{"internalType": "uint256","name": "nonce","type": "uint256"},{"internalType": "uint256","name": "expiry","type": "uint256"},{"internalType": "uint8","name": "v","type": "uint8"},{"internalType": "bytes32","name": "r","type": "bytes32"},{"internalType": "bytes32","name": "s","type": "bytes32"}],"name": "selfPermitAllowed","outputs": [],"stateMutability": "payable","type": "function"},{"inputs": [{"internalType": "address","name": "token","type": "address"},{"internalType": "uint256","name": "nonce","type": "uint256"},{"internalType": "uint256","name": "expiry","type": "uint256"},{"internalType": "uint8","name": "v","type": "uint8"},{"internalType": "bytes32","name": "r","type": "bytes32"},{"internalType": "bytes32","name": "s","type": "bytes32"}],"name": "selfPermitAllowedIfNecessary","outputs": [],"stateMutability": "payable","type": "function"},{"inputs": [{"internalType": "address","name": "token","type": "address"},{"internalType": "uint256","name": "value","type": "uint256"},{"internalType": "uint256","name": "deadline","type": "uint256"},{"internalType": "uint8","name": "v","type": "uint8"},{"internalType": "bytes32","name": "r","type": "bytes32"},{"internalType": "bytes32","name": "s","type": "bytes32"}],"name": "selfPermitIfNecessary","outputs": [],"stateMutability": "payable","type": "function"},{"inputs": [{"internalType": "address","name": "operator","type": "address"},{"internalType": "bool","name": "approved","type": "bool"}],"name": "setApprovalForAll","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "bytes4","name": "interfaceId","type": "bytes4"}],"name": "supportsInterface","outputs": [{"internalType": "bool","name": "","type": "bool"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "address","name": "token","type": "address"},{"internalType": "uint256","name": "amountMinimum","type": "uint256"},{"internalType": "address","name": "recipient","type": "address"}],"name": "sweepToken","outputs": [],"stateMutability": "payable","type": "function"},{"inputs": [],"name": "symbol","outputs": [{"internalType": "string","name": "","type": "string"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "uint256","name": "index","type": "uint256"}],"name": "tokenByIndex","outputs": [{"internalType": "uint256","name": "","type": "uint256"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "address","name": "owner","type": "address"},{"internalType": "uint256","name": "index","type": "uint256"}],"name": "tokenOfOwnerByIndex","outputs": [{"internalType": "uint256","name": "","type": "uint256"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "uint256","name": "tokenId","type": "uint256"}],"name": "tokenURI","outputs": [{"internalType": "string","name": "","type": "string"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "totalSupply","outputs": [{"internalType": "uint256","name": "","type": "uint256"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "address","name": "from","type": "address"},{"internalType": "address","name": "to","type": "address"},{"internalType": "uint256","name": "tokenId","type": "uint256"}],"name": "transferFrom","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "uint256","name": "amount0Owed","type": "uint256"},{"internalType": "uint256","name": "amount1Owed","type": "uint256"},{"internalType": "bytes","name": "data","type": "bytes"}],"name": "uniswapV3MintCallback","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "uint256","name": "amountMinimum","type": "uint256"},{"internalType": "address","name": "recipient","type": "address"}],"name": "unwrapWETH9","outputs": [],"stateMutability": "payable","type": "function"},{"stateMutability": "payable","type": "receive"}]'
    factoryABI = '[{"inputs": [],"stateMutability": "nonpayable","type": "constructor"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "uint24","name": "fee","type": "uint24"},{"indexed": true,"internalType": "int24","name": "tickSpacing","type": "int24"}],"name": "FeeAmountEnabled","type": "event"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "address","name": "oldOwner","type": "address"},{"indexed": true,"internalType": "address","name": "newOwner","type": "address"}],"name": "OwnerChanged","type": "event"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "address","name": "token0","type": "address"},{"indexed": true,"internalType": "address","name": "token1","type": "address"},{"indexed": true,"internalType": "uint24","name": "fee","type": "uint24"},{"indexed": false,"internalType": "int24","name": "tickSpacing","type": "int24"},{"indexed": false,"internalType": "address","name": "pool","type": "address"}],"name": "PoolCreated","type": "event"},{"inputs": [{"internalType": "address","name": "tokenA","type": "address"},{"internalType": "address","name": "tokenB","type": "address"},{"internalType": "uint24","name": "fee","type": "uint24"}],"name": "createPool","outputs": [{"internalType": "address","name": "pool","type": "address"}],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "uint24","name": "fee","type": "uint24"},{"internalType": "int24","name": "tickSpacing","type": "int24"}],"name": "enableFeeAmount","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "uint24","name": "","type": "uint24"}],"name": "feeAmountTickSpacing","outputs": [{"internalType": "int24","name": "","type": "int24"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "address","name": "","type": "address"},{"internalType": "address","name": "","type": "address"},{"internalType": "uint24","name": "","type": "uint24"}],"name": "getPool","outputs": [{"internalType": "address","name": "","type": "address"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "owner","outputs": [{"internalType": "address","name": "","type": "address"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "parameters","outputs": [{"internalType": "address","name": "factory","type": "address"},{"internalType": "address","name": "token0","type": "address"},{"internalType": "address","name": "token1","type": "address"},{"internalType": "uint24","name": "fee","type": "uint24"},{"internalType": "int24","name": "tickSpacing","type": "int24"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "address","name": "_owner","type": "address"}],"name": "setOwner","outputs": [],"stateMutability": "nonpayable","type": "function"}]'
    poolABI = '[{"inputs": [],"stateMutability": "nonpayable","type": "constructor"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "address","name": "owner","type": "address"},{"indexed": true,"internalType": "int24","name": "tickLower","type": "int24"},{"indexed": true,"internalType": "int24","name": "tickUpper","type": "int24"},{"indexed": false,"internalType": "uint128","name": "amount","type": "uint128"},{"indexed": false,"internalType": "uint256","name": "amount0","type": "uint256"},{"indexed": false,"internalType": "uint256","name": "amount1","type": "uint256"}],"name": "Burn","type": "event"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "address","name": "owner","type": "address"},{"indexed": false,"internalType": "address","name": "recipient","type": "address"},{"indexed": true,"internalType": "int24","name": "tickLower","type": "int24"},{"indexed": true,"internalType": "int24","name": "tickUpper","type": "int24"},{"indexed": false,"internalType": "uint128","name": "amount0","type": "uint128"},{"indexed": false,"internalType": "uint128","name": "amount1","type": "uint128"}],"name": "Collect","type": "event"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "address","name": "sender","type": "address"},{"indexed": true,"internalType": "address","name": "recipient","type": "address"},{"indexed": false,"internalType": "uint128","name": "amount0","type": "uint128"},{"indexed": false,"internalType": "uint128","name": "amount1","type": "uint128"}],"name": "CollectProtocol","type": "event"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "address","name": "sender","type": "address"},{"indexed": true,"internalType": "address","name": "recipient","type": "address"},{"indexed": false,"internalType": "uint256","name": "amount0","type": "uint256"},{"indexed": false,"internalType": "uint256","name": "amount1","type": "uint256"},{"indexed": false,"internalType": "uint256","name": "paid0","type": "uint256"},{"indexed": false,"internalType": "uint256","name": "paid1","type": "uint256"}],"name": "Flash","type": "event"},{"anonymous": false,"inputs": [{"indexed": false,"internalType": "uint16","name": "observationCardinalityNextOld","type": "uint16"},{"indexed": false,"internalType": "uint16","name": "observationCardinalityNextNew","type": "uint16"}],"name": "IncreaseObservationCardinalityNext","type": "event"},{"anonymous": false,"inputs": [{"indexed": false,"internalType": "uint160","name": "sqrtPriceX96","type": "uint160"},{"indexed": false,"internalType": "int24","name": "tick","type": "int24"}],"name": "Initialize","type": "event"},{"anonymous": false,"inputs": [{"indexed": false,"internalType": "address","name": "sender","type": "address"},{"indexed": true,"internalType": "address","name": "owner","type": "address"},{"indexed": true,"internalType": "int24","name": "tickLower","type": "int24"},{"indexed": true,"internalType": "int24","name": "tickUpper","type": "int24"},{"indexed": false,"internalType": "uint128","name": "amount","type": "uint128"},{"indexed": false,"internalType": "uint256","name": "amount0","type": "uint256"},{"indexed": false,"internalType": "uint256","name": "amount1","type": "uint256"}],"name": "Mint","type": "event"},{"anonymous": false,"inputs": [{"indexed": false,"internalType": "uint8","name": "feeProtocol0Old","type": "uint8"},{"indexed": false,"internalType": "uint8","name": "feeProtocol1Old","type": "uint8"},{"indexed": false,"internalType": "uint8","name": "feeProtocol0New","type": "uint8"},{"indexed": false,"internalType": "uint8","name": "feeProtocol1New","type": "uint8"}],"name": "SetFeeProtocol","type": "event"},{"anonymous": false,"inputs": [{"indexed": true,"internalType": "address","name": "sender","type": "address"},{"indexed": true,"internalType": "address","name": "recipient","type": "address"},{"indexed": false,"internalType": "int256","name": "amount0","type": "int256"},{"indexed": false,"internalType": "int256","name": "amount1","type": "int256"},{"indexed": false,"internalType": "uint160","name": "sqrtPriceX96","type": "uint160"},{"indexed": false,"internalType": "uint128","name": "liquidity","type": "uint128"},{"indexed": false,"internalType": "int24","name": "tick","type": "int24"}],"name": "Swap","type": "event"},{"inputs": [{"internalType": "int24","name": "tickLower","type": "int24"},{"internalType": "int24","name": "tickUpper","type": "int24"},{"internalType": "uint128","name": "amount","type": "uint128"}],"name": "burn","outputs": [{"internalType": "uint256","name": "amount0","type": "uint256"},{"internalType": "uint256","name": "amount1","type": "uint256"}],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "address","name": "recipient","type": "address"},{"internalType": "int24","name": "tickLower","type": "int24"},{"internalType": "int24","name": "tickUpper","type": "int24"},{"internalType": "uint128","name": "amount0Requested","type": "uint128"},{"internalType": "uint128","name": "amount1Requested","type": "uint128"}],"name": "collect","outputs": [{"internalType": "uint128","name": "amount0","type": "uint128"},{"internalType": "uint128","name": "amount1","type": "uint128"}],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "address","name": "recipient","type": "address"},{"internalType": "uint128","name": "amount0Requested","type": "uint128"},{"internalType": "uint128","name": "amount1Requested","type": "uint128"}],"name": "collectProtocol","outputs": [{"internalType": "uint128","name": "amount0","type": "uint128"},{"internalType": "uint128","name": "amount1","type": "uint128"}],"stateMutability": "nonpayable","type": "function"},{"inputs": [],"name": "factory","outputs": [{"internalType": "address","name": "","type": "address"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "fee","outputs": [{"internalType": "uint24","name": "","type": "uint24"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "feeGrowthGlobal0X128","outputs": [{"internalType": "uint256","name": "","type": "uint256"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "feeGrowthGlobal1X128","outputs": [{"internalType": "uint256","name": "","type": "uint256"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "address","name": "recipient","type": "address"},{"internalType": "uint256","name": "amount0","type": "uint256"},{"internalType": "uint256","name": "amount1","type": "uint256"},{"internalType": "bytes","name": "data","type": "bytes"}],"name": "flash","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "uint16","name": "observationCardinalityNext","type": "uint16"}],"name": "increaseObservationCardinalityNext","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "uint160","name": "sqrtPriceX96","type": "uint160"}],"name": "initialize","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [],"name": "liquidity","outputs": [{"internalType": "uint128","name": "","type": "uint128"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "maxLiquidityPerTick","outputs": [{"internalType": "uint128","name": "","type": "uint128"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "address","name": "recipient","type": "address"},{"internalType": "int24","name": "tickLower","type": "int24"},{"internalType": "int24","name": "tickUpper","type": "int24"},{"internalType": "uint128","name": "amount","type": "uint128"},{"internalType": "bytes","name": "data","type": "bytes"}],"name": "mint","outputs": [{"internalType": "uint256","name": "amount0","type": "uint256"},{"internalType": "uint256","name": "amount1","type": "uint256"}],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "uint256","name": "","type": "uint256"}],"name": "observations","outputs": [{"internalType": "uint32","name": "blockTimestamp","type": "uint32"},{"internalType": "int56","name": "tickCumulative","type": "int56"},{"internalType": "uint160","name": "secondsPerLiquidityCumulativeX128","type": "uint160"},{"internalType": "bool","name": "initialized","type": "bool"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "uint32[]","name": "secondsAgos","type": "uint32[]"}],"name": "observe","outputs": [{"internalType": "int56[]","name": "tickCumulatives","type": "int56[]"},{"internalType": "uint160[]","name": "secondsPerLiquidityCumulativeX128s","type": "uint160[]"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "bytes32","name": "","type": "bytes32"}],"name": "positions","outputs": [{"internalType": "uint128","name": "liquidity","type": "uint128"},{"internalType": "uint256","name": "feeGrowthInside0LastX128","type": "uint256"},{"internalType": "uint256","name": "feeGrowthInside1LastX128","type": "uint256"},{"internalType": "uint128","name": "tokensOwed0","type": "uint128"},{"internalType": "uint128","name": "tokensOwed1","type": "uint128"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "protocolFees","outputs": [{"internalType": "uint128","name": "token0","type": "uint128"},{"internalType": "uint128","name": "token1","type": "uint128"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "uint8","name": "feeProtocol0","type": "uint8"},{"internalType": "uint8","name": "feeProtocol1","type": "uint8"}],"name": "setFeeProtocol","outputs": [],"stateMutability": "nonpayable","type": "function"},{"inputs": [],"name": "slot0","outputs": [{"internalType": "uint160","name": "sqrtPriceX96","type": "uint160"},{"internalType": "int24","name": "tick","type": "int24"},{"internalType": "uint16","name": "observationIndex","type": "uint16"},{"internalType": "uint16","name": "observationCardinality","type": "uint16"},{"internalType": "uint16","name": "observationCardinalityNext","type": "uint16"},{"internalType": "uint8","name": "feeProtocol","type": "uint8"},{"internalType": "bool","name": "unlocked","type": "bool"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "int24","name": "tickLower","type": "int24"},{"internalType": "int24","name": "tickUpper","type": "int24"}],"name": "snapshotCumulativesInside","outputs": [{"internalType": "int56","name": "tickCumulativeInside","type": "int56"},{"internalType": "uint160","name": "secondsPerLiquidityInsideX128","type": "uint160"},{"internalType": "uint32","name": "secondsInside","type": "uint32"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "address","name": "recipient","type": "address"},{"internalType": "bool","name": "zeroForOne","type": "bool"},{"internalType": "int256","name": "amountSpecified","type": "int256"},{"internalType": "uint160","name": "sqrtPriceLimitX96","type": "uint160"},{"internalType": "bytes","name": "data","type": "bytes"}],"name": "swap","outputs": [{"internalType": "int256","name": "amount0","type": "int256"},{"internalType": "int256","name": "amount1","type": "int256"}],"stateMutability": "nonpayable","type": "function"},{"inputs": [{"internalType": "int16","name": "","type": "int16"}],"name": "tickBitmap","outputs": [{"internalType": "uint256","name": "","type": "uint256"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "tickSpacing","outputs": [{"internalType": "int24","name": "","type": "int24"}],"stateMutability": "view","type": "function"},{"inputs": [{"internalType": "int24","name": "","type": "int24"}],"name": "ticks","outputs": [{"internalType": "uint128","name": "liquidityGross","type": "uint128"},{"internalType": "int128","name": "liquidityNet","type": "int128"},{"internalType": "uint256","name": "feeGrowthOutside0X128","type": "uint256"},{"internalType": "uint256","name": "feeGrowthOutside1X128","type": "uint256"},{"internalType": "int56","name": "tickCumulativeOutside","type": "int56"},{"internalType": "uint160","name": "secondsPerLiquidityOutsideX128","type": "uint160"},{"internalType": "uint32","name": "secondsOutside","type": "uint32"},{"internalType": "bool","name": "initialized","type": "bool"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "token0","outputs": [{"internalType": "address","name": "","type": "address"}],"stateMutability": "view","type": "function"},{"inputs": [],"name": "token1","outputs": [{"internalType": "address","name": "","type": "address"}],"stateMutability": "view","type": "function"}]'
    
    def __init__(self, wallet_address: str, private_key: str, node: str, **kwargs):
        super().__init__(wallet_address, private_key, node)
        self.w3 = Web3(Web3.HTTPProvider(node))
        self.contract = self.w3.eth.contract('0x10ED43C718714eb63d5aA57B78B54704E256024E', abi=self.ABI)

    def get_position(self):
        nonFungiblePositionManager = self.w3.eth.contract(
            address='0xC36442b4a4522E871399CD717aBDD847Ab11FE88', 
            abi=nonFungiblePositionManagerABI
        )
        return nonFungiblePositionManager.functions.positions().call()
    
    def _prepare_ticker(self, ticker: str) -> (int, list):
        symbol0, symbol1 = ticker.split('/')
        token0 = symbol_to_token(symbol0)
        token1 = symbol_to_token(symbol1)
        return token0, token1
    
    def get_price(self, ticker: str, quantity: float, **kwargs) -> float:
        token0, token1 = self._prepare_ticker(ticker)
        _quantity = quantity * 10**token0.decimals
        path = [token0.address, token1.address, 3000]
        factoryContract = self.w3.eth.contract(
            address='0x1F98431c8aD98523631AE4a59f267346ea31F984', 
            abi=factoryABI
        )
        poolAdress = factoryContract.functions.getPool(*path).call()
        pool_contract = self.w3.eth.contract(address=poolAdress, abi=poolABI)
        
        t0 = pool_contract.functions.token0().call()
        t1 = pool_contract.functions.token1().call()
        if t1.lower() == token0.lower():
            den0 = self.get_token(token0).decimals
            den1 = self.get_token(token1).decimals
        else:
            den0 = self.get_token(token1).decimals
            den1 = self.get_token(token0).decimals
        sqrtPriceX96 = pool_contract.functions.slot0().call()[0]
        raw_price = (sqrtPriceX96 * sqrtPriceX96 * 10**den1 >> (96 * 2)) / (
            10**den0
        )
        if t1.lower() == token0.lower():
            raw_price = 1 / raw_price
        return raw_price